# Q&A against Tabular Data from a CSV file  (experimental)

Smart Search Engine이 모든 문서에 대한 질문에 대답할 수 있으려면,  표 형태의 데이터, 즉 행 및 열의 출처와 숫자를 이해해야 합니다.
이것은 단순히 가장 유사한 상위 결과를 찾는 것과는 다른 문제입니다. 인덱싱의 개념, 상위 결과 가져오기, 임베딩, 코사인 의미 검색 및 답변 요약의 개념은 표 형태의 데이터에는 영향을 미치지 않습니다. 
그렇기에 각 행과 열이 서로 연관되어 있는 표를 기반으로, 질문에 답하기 위해서는 상위 결과만이 아니라 모든 데이터가 필요합니다.

데이터 레이크에 있는 많은 문서들이 표 형태의 데이터입니다. 
이런 LLM모델을 사용하여 표 형태의 데이터를 이해하고 질문에 대답하기 위해서 CSV 파일이나 심지어 SQL 데이터베이스에서 문맥을 얻어야 합니다. 스마트 검색 엔진은 이러한 소스를 다루는 방법을 이해하고, 데이터를 분석하여, 질문에 적합한 답을 도출합니다. 

In [4]:
# pip install gdown

In [5]:
import os
import pandas as pd
import gdown
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents import create_csv_agent

from common.prompts import CSV_PROMPT_PREFIX, CSV_PROMPT_SUFFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))


In [6]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

# Download the dataset and load it into Pandas Dataframe

In [7]:
os.makedirs("data",exist_ok=True)

In [8]:
# !wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/

In [9]:
file_id = "1bXIucVZHBUKyj0cF5iBXNfqUQYzLtiaS"
file_url = "./data/Seoul_Commercial_Districts.csv" # 저장 위치 및 저장할 파일 이름
gdown.download(id=file_id, output=file_url, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1bXIucVZHBUKyj0cF5iBXNfqUQYzLtiaS
To: c:\Projects\ClooSearch-HOL\ClooSearch-HOL\data\Seoul_Commercial_Districts.csv
100%|██████████| 229k/229k [00:00<00:00, 1.03MB/s]


'./data/Seoul_Commercial_Districts.csv'

In [10]:
# file_url = "./data/"
df = pd.read_csv(file_url, encoding='cp949').fillna(value = 0)
print("Rows and Columns:",df.shape)
df.head()

Rows and Columns: (3455, 8)


,기준분기,상권구분,상권지역,서비스업종,분기매출,분기당_매출건수,주중_매출금액,주말_매출금액
0,4,발달상권,대림3동사거리,가구,205619535.0,220,205619535.0,0
1,4,발달상권,송파사거리(송파역),시계및귀금속,47289011.0,156,47289011.0,0
2,4,발달상권,상계백병원,가구,3509705.0,11,3509705.0,0
3,4,전통시장,화곡본동시장,가구,6554404.0,10,6554404.0,0
4,4,발달상권,강남을지병원,가구,125073744.0,108,125073744.0,0


In [11]:
df.columns

Index(['기준분기', '상권구분', '상권지역', '서비스업종', '분기매출', '분기당_매출건수', '주중_매출금액',
       '주말_매출금액'],
      dtype='object')

## Introducing: Agents

스마트 에이전트의 구현은 [MRKL Systems](https://arxiv.org/abs/2205.00445)논문과 [ReAct](https://arxiv.org/abs/2210.03629)논문에서 영감을 얻어 만들어졌습니다.

스마트 에이전트는 LLM이 프롬프트를 이해하고 행동하는 능력을 활용하는 기술입니다. 기본적으로 스마트 에이전트는 프롬프트를 사용하여LLM에게 복잡한 쿼리를 하나씩 일련의 단계로 분해하여 응답하는 과정을 거칩니다. 

Agent에 NRKL 논문에서 소개한 'expert'를 결합시키면 더욱 똑똑해집니다. 간단한 예시로 Agent는 스스로 수학적 계산을 안정적으로 수행하는 고유한 기능이 없을 수 있습니다. 그러나 이러한 경우에서 수학적 계산기인 'expert'를 소개하겠습니다. 이제 우리가 계산을 해야할 때, Agent는 결과 자체를 직접 예측하기 보다는 'expert'를 불러올 수 있습니다. 이것은 [ChatGPT Pluggins](https://openai.com/blog/chatgpt-plugins)의 배경이 되는 개념입니다.

OpenAI는 새로운 개념으로 세상을 새롭게 열었습니다. 라이브러리들은 빠르고 격렬하게 만들어지고 있습니다. 우리는 문제를 해결하기 위하여 [LangChain](https://docs.langchain.com/docs/)이라는 라이브러리를 사용할 예정이지만 추천하는 다른 라이브러리도 있습니다. (예: [HayStack](https://haystack.deepset.ai/), [Semantic Kernel](https://learn.microsoft.com/en-us/semantic-kernel/whatissk))

In [12]:
# 앞서 설명한 것과 같이 일련의 단계를 해결하기 위해 질문을 짚어보겠습니다. 당장은 질문의 답이 정확하지 않을 수 있습니다. 

QUESTION = "발달상권 가구의 4분기 분기매출을 알려주고 주중과 주말로 나눠서도 매출을 알려줘"


In [13]:
# GPT-4 모델을 사용합니다.
# GPT-3.5-Turbo는 복잡한 문제를 해결할 수 있는 인지기능이 없기에 부정확한 답을 줄 수 있습니다. 
llm = AzureChatOpenAI(deployment_name="gpt-4", temperature=0, max_tokens=500)

LangChain의 CSV 표 형식의 데이터을 이해하고 질문을 해결하기 위해 사용할 수 있는 out-of-box 에이전트를 사용합니다. **CSV 에이전트**에 대한 자세한 설명은 [이곳](https://python.langchain.com/en/latest/modules/agents/toolkits/examples/csv.html)을 클릭하십시오

In [14]:
agent_executor = create_pandas_dataframe_agent(llm=llm,df=df,verbose=True)

In [15]:
agent_executor.agent.allowed_tools

['python_repl_ast']

In [16]:
printmd(agent_executor.agent.llm_chain.prompt.template)


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df_head}

Begin!
Question: {input}
{agent_scratchpad}

## Enjoy the response and the power of GPT-4 + REACT/MKRL approach

## GPT-4 + REACT/MKRL 접근방식의 뛰어난 성능을 즐겨보세요!

In [17]:
# for 문을 사용하여 루프를 돌립니다. 그 이유는:
# 1) GPT-4 는 아직 preview 단계이고
# 2) LLM 모델이 에이전트가 필요로 하는 정확한 형식에 대한 답변을 항상 제공하는 것은 아니기 때문입니다. 

for i in range(5):
    try:
        response = agent_executor.run(CSV_PROMPT_PREFIX + QUESTION + CSV_PROMPT_SUFFIX) 
        break
    except:
        response = "Error too many failed retries"
        continue
        
print(response)



> Entering new AgentExecutor chain...
Thought: I need to set the pandas display options to show all the columns first.
Action: python_repl_ast
Action Input: pd.set_option('display.max_columns', None)
Observation: NameError: name 'pd' is not defined
Thought:I forgot to import pandas. I need to import pandas first.
Action: python_repl_ast
Action Input: import pandas as pd
Observation: 
Thought:Now that pandas is imported, I can set the display options to show all the columns.
Action: python_repl_ast
Action Input: pd.set_option('display.max_columns', None)
Observation: 
Thought:Now that I have set the display options, I can get the column names.
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['기준분기', '상권구분', '상권지역', '서비스업종', '분기매출', '분기당_매출건수', '주중_매출금액',
       '주말_매출금액'],
      dtype='object')
Thought:The column names are '기준분기', '상권구분', '상권지역', '서비스업종', '분기매출', '분기당_매출건수', '주중_매출금액', and '주말_매출금액'. Now I need to filter the dataframe for rows where '상권구분' is '발달상권

## 평가
정답이 맞는지 확인해 보겠습니다. 

In [18]:
sales_of_furniture = df[(df['기준분기'] == 4) & (df['상권구분'] == '발달상권') & (df['서비스업종'] == '가구')]
week_sales = sales_of_furniture['주중_매출금액'].sum()
weekend_sales = sales_of_furniture['주말_매출금액'].sum()
total_sales_of_furniture= sales_of_furniture['분기매출'].sum()

print( "4분기 가구 전채매출: ",total_sales_of_furniture,"4분기 가구 주중매출: ",week_sales, "4분기 가구 주말매출: ", weekend_sales )

4분기 가구 전채매출:  127152344979.0 4분기 가구 주중매출:  83906665297.0 4분기 가구 주말매출:  43245679682


**Note**: 위의 셀을 여러 번 실행하면 항상 같은 결과가 나오는 것이 아니며 때로는 오류가 발생하기도 합니다. 그 이유는: 
1) LLM과 라이브러리는 아직 새로운 분야이고 여전히 성장할 여지가 많습니다.
2) 여러 단계로 풀어야 하는 복잡한 문제는 사람도 실수를 하기 때문입니다.
3) 열 이름이 명확하지 않거나 모호한 데이터 있으면 사람이 하는 것처럼 실수를 저지를 수 있기 때문입니다.

# Summary

CSV 파일로 호스팅 된 우리의 Tabular data에 자연어로 질문하는 방법에 대한 문제를 해결하였습니다.
이 접근 방식을 사용하면 데이터베이스 데이터를 CSV 파일로 덤프하고 검색 엔진 위에서 위의 접근 방식을 사용하여 CSV 데이터 덤프 파일을 처리할 필요가 없다는 것을 알 수 있습니다. Agent Framework를 사용하면 최상의 엔지니어링 결정은 데이터 소스 API에 질문을 하기 위해 데이터를 복제할 필요 없이 데이터 소스 API와 직접 상호 작용하는 것입니다.
GPT-4는 SQL을 매우 잘 수행할 수 있습니다.

GPT-4가 이러한 것들을 훌륭하게 해낼 수 있다면, GPT의 후속 모델들은 어떠한 기능을 수행할 수 있을지 상상해보세요!

**Note**: tabular data의 질문에 답하기 위해 Pandas agent를 사용하지 않는 것이 좋습니다. 속도가 빠르지 않고 구문 분석 실수를 많이 하기 때문입니다. 이보단 SQL을 이용하는 것이 더 좋습니다.(다음 노트북을 참조하세요)

# Reference

- https://haystack.deepset.ai/blog/introducing-haystack-agents
- https://python.langchain.com/en/latest/modules/agents/agents.html
- https://tsmatz.wordpress.com/2023/03/07/react-with-openai-gpt-and-langchain/
- https://medium.com/@meghanheintz/intro-to-langchain-llm-templates-and-agents-8793f30f1837